# Setup

## imports

In [26]:
import time
import os
import json
import glob
import re
import requests
import xml.etree.ElementTree as ET
import datetime
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

## base helper methods

In [2]:
def find_file_with_wildcard(pattern):
    """Find file with wildcard pattern, raise error if no match or multiple matches."""
    # Use glob to find matching files
    matching_files = glob.glob(pattern)
    
    # Check if no file was found
    if len(matching_files) == 0:
        raise FileNotFoundError(f"No files found matching pattern: {pattern}")
    
    # Check if more than one file was found
    if len(matching_files) > 1:
        raise ValueError(f"Multiple files found matching pattern: {pattern}")
    
    # Return the matched file path
    return matching_files[0]

## base vars

In [3]:
# Define scopes for accessing Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']
MEMORY_FILE = 'processed_files.json'
CREDENTIALS_FILE_WILDCARDED = './.credentials/client_secret*.json'
CREDENTIALS_FILE = find_file_with_wildcard(CREDENTIALS_FILE_WILDCARDED)
TOKEN_FILE = './.credentials/token.json'
SERVER_FOLDER = "server_files"

## drive helper methods

In [4]:
def authenticate_google_drive():
    """Authenticate with Google Drive and return the service object."""
    creds = None
    # Check if token.json exists (token storage for authenticated user)
    if os.path.exists(TOKEN_FILE):
        print("Loading credentials from token.json")
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    
    # Authenticate if credentials are not valid or do not exist
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("Refreshing expired credentials")
            creds.refresh(Request())
        else:
            print("Authenticating with Google Drive")
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save credentials for the next run
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
    
    # Build the Drive API service
    return build('drive', 'v3', credentials=creds)

def get_items_by_name(service, name, is_folder=False, is_root=False):
    """Get a list of items by name, including their modified time, sorted by modified time."""
    try:
        q = f"name = '{name}'"
        if is_folder:
            q += " and mimeType = 'application/vnd.google-apps.folder'"
        else:
            q += " and mimeType != 'application/vnd.google-apps.folder'"
        if is_root:
            q += " and 'root' in parents"

        results = service.files().list(
            q=q,
            fields="nextPageToken, files(id, name, mimeType, modifiedTime, parents)",
            orderBy="modifiedTime desc"
        ).execute()
        return results.get('files', [])
    except HttpError as error:
        print(f"An error occurred: {error}")
        return []

def move_file_to_folder(service, file_id, folder_id):
    """Move a file to a specific folder in Google Drive."""
    try:
        # Retrieve the current parents
        file = service.files().get(fileId=file_id, fields='parents').execute()
        previous_parents = ",".join(file.get('parents', []))

        # Move the file to the new folder
        service.files().update(
            fileId=file_id,
            addParents=folder_id,
            removeParents=previous_parents,
            fields='id, parents'
        ).execute()
        print(f"File {file_id} has been moved to folder {folder_id}.")
    except HttpError as error:
        print(f"An error occurred while moving the file: {error}")

## drive variables

In [5]:
SERVICE = authenticate_google_drive()
FLASCHARD_FILE_ARCHIVE_FOLDER = "_Pleco"
FLASCHARD_FILE_ARCHIVE_FOLDER_ID = get_items_by_name(SERVICE, FLASCHARD_FILE_ARCHIVE_FOLDER, is_folder=True, is_root=True)[0]['id']
FLASHCARD_FILE_NAME = "pleco_flashcards.xml"

Loading credentials from token.json
Refreshing expired credentials


## anki connect variables

In [7]:
# AnkiConnect API endpoint
ANKI_CONNECT_URL = "http://localhost:8765"


## anki connect methods

In [11]:
def get_latest_anki_flaschard_words():
    flashcard_set = set()

    # Step 0: Sync Anki with the cloud
    sync_payload = {
        "action": "sync",
        "version": 6
    }

    sync_response = requests.post(ANKI_CONNECT_URL, json=sync_payload)

    if sync_response.status_code == 200:
        sync_result = sync_response.json()
        if sync_result.get('error'):
            raise ConnectionError(f"Error syncing Anki: {sync_result['error']}")
        else:
            print("Anki sync successful.")
    else:
        raise ConnectionError("Failed to connect to AnkiConnect for syncing.")
    

    # Step 1: Find all card IDs in the "Pleco Import" deck
    find_payload = {
        "action": "findCards",
        "version": 6,
        "params": {
            "query": 'deck:"Pleco Import"'  # Query for cards only in the "Pleco Import" deck
        }
    }

    response = requests.post(ANKI_CONNECT_URL, json=find_payload)

    if response.status_code == 200:
        find_result = response.json()
        if 'result' in find_result and find_result['result']:
            card_ids = find_result['result']
            print(f"Total cards found in 'Pleco Import': {len(card_ids)}")

            # Step 2: Retrieve detailed information for these card IDs
            info_payload = {
                "action": "cardsInfo",
                "version": 6,
                "params": {
                    "cards": card_ids  # Pass the list of card IDs
                }
            }

            info_response = requests.post(ANKI_CONNECT_URL, json=info_payload)

            if info_response.status_code == 200:
                info_result = info_response.json()
                if 'result' in info_result and info_result['result']:
                    # Step 3: Extract the "Front" field for each card and add to the set
                    for card in info_result['result']:
                        card_id = card['cardId']
                        front_field = re.sub(r'[^\u4e00-\u9fff]', '', card['fields']['Front']['value'])  # Filter out non-Chinese characters
                        flashcard_set.add(front_field)
                    print("success")
                else:
                    raise ConnectionError(f"Error retrieving card information: {info_result.get('error')}")
            else:
                raise ConnectionError("Failed to connect to AnkiConnect for card information.")
        else:
            raise ValueError(f"No cards found or error: {find_result.get('error')}")
    else:
        raise ConnectionError("Failed to connect to AnkiConnect for finding cards.")

    return flashcard_set


# Main

In [29]:
# main google drive functions
def get_latest_flashcard_xml():
    files = get_items_by_name(SERVICE, FLASHCARD_FILE_NAME, is_folder=False, is_root=True)
    if len(files) == 0:
        print("No files found.")
        return None
        
    # Download the content of the target file
    target_file = files[0]
    request = SERVICE.files().get_media(fileId=target_file['id'])
    file_content = request.execute()
    file_text = file_content.decode('utf-8')
    modified_time = datetime.datetime.fromisoformat(target_file['modifiedTime'].replace('Z', '+00:00'))
    local_time = modified_time.astimezone()
    print("Latest flashcard xml last modified time:", local_time.strftime('%Y-%m-%d %H:%M:%S %Z%z'))
    return file_text

def archive_flashcard_xmls(archive_latest=False):
    files = get_items_by_name(SERVICE, FLASHCARD_FILE_NAME, is_folder=False, is_root=True)
    if len(files) == 0:
        print("No files found.")
        return

    # move irrelevant files to archive folder
    for f in files[0 if archive_latest else 1:]:
        move_file_to_folder(SERVICE, f['id'], FLASCHARD_FILE_ARCHIVE_FOLDER_ID)

In [18]:
# main flashcard processing functions
def convert_pinyin(pinyin):
    tone_marks = {
        'a': 'āáǎàa',
        'e': 'ēéěèe',
        'i': 'īíǐìi',
        'o': 'ōóǒòo',
        'u': 'ūúǔùu',
        'ü': 'ǖǘǚǜü'
    }
    result = []
    cleaned_pinyin= re.sub(r'\s', '', pinyin)

    for syllable in re.split(r'(?<=\d)(?=\D)', cleaned_pinyin):
        if syllable[-1].isdigit():
            tone = int(syllable[-1])
            syllable = syllable[:-1]
            for vowel in 'aeiouü':
                if vowel in syllable:
                    syllable = syllable.replace(vowel, tone_marks[vowel][tone - 1])
                    break
        result.append(syllable)
    return ' '.join(result)

def process_flashcard_xml(xml_text):
    """returns a list of dictionaries with simplified, traditional, pinyin, and definition"""

    # Parse the XML content
    root = ET.fromstring(xml_text)

    # Find all <entry> tags within <card> tags
    entries = root.findall('.//card/entry')

    # Extract and print each entry
    entries_data = []
    for entry in entries:
        simplified = entry.find('headword[@charset="sc"]').text
        traditional = entry.find('headword[@charset="tc"]').text
        pinyin = convert_pinyin(entry.find('pron').text)
        definition = entry.find('defn').text
        
        entry_data = {
            'simplified': simplified,
            'traditional': traditional,
            'pinyin': pinyin,
            'definition': definition,
        }
        entries_data.append(entry_data)
    return entries_data


## extra functions

In [37]:
def list_files_by_name(service, file_name):
    """List files with a specific name from Google Drive."""
    try:
        results = service.files().list(
            q=f"name = '{file_name}'",
            fields="nextPageToken, files(id, name, mimeType, modifiedTime, parents)",
            orderBy="modifiedTime desc"
        ).execute()
        return results.get('files', [])
    except HttpError as error:
        print(f"An error occurred: {error}")
        return []

def process_file(file):
    """Placeholder function to process a file."""
    print(f"Processing file: {file['name']} (ID: {file['id']})")


def load_processed_files():
    """Load the list of processed file IDs from a local memory file."""
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, 'r') as file:
            return set(json.load(file))
    return set()

def save_processed_files(processed_files):
    """Save the list of processed file IDs to a local memory file."""
    with open(MEMORY_FILE, 'w') as file:
        json.dump(list(processed_files), file)

def monitor_google_drive(target_folder_id, interval=60):
    """Periodically check for new files, process them, and move them."""
    service = authenticate_google_drive()
    processed_files = load_processed_files()

    while True:
        print("Checking for new files named 'pleco_flashcards.xml'...")
        files = list_files_by_name(service, "pleco_flashcards.xml")

        for file in files:
            if file['id'] not in processed_files:
                # Process the file
                process_file(file)

                # Move the file to the specified folder
                move_file_to_folder(service, file['id'], target_folder_id)

                # Mark the file as processed and save to memory
                processed_files.add(file['id'])
                save_processed_files(processed_files)
        
        # Wait for the specified interval before checking again
        time.sleep(interval)


## sandbox 1

In [30]:
current_anki_flash_card_words = get_latest_anki_flaschard_words()
xml_text = get_latest_flashcard_xml()
flashcard_entries = process_flashcard_xml(xml_text)

Anki sync successful.
Total cards found in 'Pleco Import': 1659
success
Latest flashcard xml last modified time: 2025-01-01 22:53:27 EST-0500


In [31]:
for entry in flashcard_entries:
    print(entry['traditional'], entry['traditional'] in current_anki_flash_card_words)

遊戲 False
革新 False
單身狗 False
老弟 False
本事 False
一線之隔 False
白痴 False
環境 False
載具 True
竟是 True
旅館 False
絲 True


In [32]:
entry_dict = {entry['traditional']: entry for entry in flashcard_entries}

In [34]:
print(entry_dict['絲']["definition"])

noun 1 silk 2 threadlike thing 銅絲 tóngsī copper wire (蜘)蛛絲 zhīzhūsī thread spun by a spider; cobweb 把土豆切成細絲兒 bǎ tǔdòu qiē chéng xì sīr cut the potatoes into fine shreds; shred the potatoes 3 stringed instruments See 30756096絲竹si1zhu2絲竹 
measure word 1 a thread or shred of 一絲亮光 yī sī liàngguāng a thread of light 這盤菜裡一絲兒肉也見不著。 Zhè pán cài lǐ yī sīr ròu yě jiàn bu zháo. Not a single shred of meat can be found in this dish. 2 si a a traditional unit of length, equal to 0.00001 chi (市尺), and equivalent to 3.333 microns b a traditional unit of weight, equal to 0.000001 jin (市斤), and equivalent to 0.5 milligram or 0.0075 grain 3 a tiny bit; trace 一絲不差 yīsībùchà not a bit of difference 一絲風也沒有。 Yī sī fēng yě méiyǒu. There isn’t a breath of air. 她臉上沒有一絲笑容。 Tā liǎnshàng méiyǒu yī sī xiàoróng. There isn’t a trace of a smile on her face. 4 one ten-thousandth of certain units of measure See 30753024絲米si1mi3絲米


In [35]:
with open("flashcard_entries.json", "w") as f:
    json.dump(flashcard_entries, f, ensure_ascii=False, indent=4)  

# sandbox

In [65]:

print(file_text)


File 1iiAdE90Oi8o8W4Won4wqFJqzm48-o0GA has been moved to folder 1S_r1Y_uGNmwlYmqByyjVa1pnQVC07Plf.
<?xml version="1.0" encoding="UTF-8"?><plecoflash formatversion="2" creator="Pleco User 19097293" generator="Pleco 2.0 Flashcard Exporter" platform="iPhone OS" created="1735693200"><categories></categories><cards><card language="chinese" created="1735513394" modified="1735513394"><entry><headword charset="sc">游戏</headword><headword charset="tc">遊戲</headword><pron type="hypy" tones="numbers">you2xi4</pron><defn>noun recreation; game 做遊戲 Zuò yóuxì play games 
verb play 孩子們在公園裡遊戲。 Háizi men zài gōngyuán lǐ yóuxì. The children are playing in the park.</defn></entry><dictref dictid="PACE" entryid="35050240"/></card><card language="chinese" created="1735514285" modified="1735514285"><entry><headword charset="sc">革新</headword><headword charset="tc">革新</headword><pron type="hypy" tones="numbers">ge2xin1</pron><defn>noun innovation; renovation 技術革新 jìshù géxīn technological innovation 
verb innova

In [93]:
def convert_pinyin(pinyin):
    tone_marks = {
        'a': 'āáǎàa',
        'e': 'ēéěèe',
        'i': 'īíǐìi',
        'o': 'ōóǒòo',
        'u': 'ūúǔùu',
        'ü': 'ǖǘǚǜü'
    }
    result = []
    cleaned_pinyin= re.sub(r'\s', '', pinyin)

    for syllable in re.split(r'(?<=\d)(?=\D)', cleaned_pinyin):
        if syllable[-1].isdigit():
            tone = int(syllable[-1])
            syllable = syllable[:-1]
            for vowel in 'aeiouü':
                if vowel in syllable:
                    syllable = syllable.replace(vowel, tone_marks[vowel][tone - 1])
                    break
        result.append(syllable)
    return ' '.join(result)

In [89]:
for vowel in 'aeiouü':
    print(vowel in "lü3guan3")

True
False
False
False
True
True


In [87]:
"ü" in "lü3guan3"

True

In [91]:
convert_pinyin("lü3guan3")

ü
a


'lǚ guǎn'

In [64]:
import re

convert_pinyin("ni2hao3")
# Split the string by the number
pinyin="ni2hao3"
split_pinyin = re.split(r'(?<=\d)(?=\D)', cleaned_pinyin)
split_pinyin

['ni2', 'hao3']

In [69]:
len(entries)

11

In [95]:

# Parse the XML content
root = ET.fromstring(file_text)

# Find all <entry> tags within <card> tags
entries = root.findall('.//card/entry')

# Extract and print each entry
for entry in entries:
    print(ET.tostring(entry, encoding='unicode'))
    # Helper function to convert pinyin with numbers to pinyin with tone marks

    # Process each entry into an object
    entries_data = []
    for entry in entries:
        simplified = entry.find('headword[@charset="sc"]').text
        traditional = entry.find('headword[@charset="tc"]').text
        pinyin = convert_pinyin(entry.find('pron').text)
        definition = entry.find('defn').text
        
        entry_data = {
            'simplified': simplified,
            'traditional': traditional,
            'pinyin': pinyin,
            'definition': definition,
        }
        entries_data.append(entry_data)

print(entries_data)

<entry><headword charset="sc">游戏</headword><headword charset="tc">遊戲</headword><pron type="hypy" tones="numbers">you2xi4</pron><defn>noun recreation; game 做遊戲 Zuò yóuxì play games 
verb play 孩子們在公園裡遊戲。 Háizi men zài gōngyuán lǐ yóuxì. The children are playing in the park.</defn></entry>
<entry><headword charset="sc">革新</headword><headword charset="tc">革新</headword><pron type="hypy" tones="numbers">ge2xin1</pron><defn>noun innovation; renovation 技術革新 jìshù géxīn technological innovation 
verb innovate; improve 傳統的手工藝技術不斷革新。 Chuántǒng de shǒu gōngyì jìshù bùduàn géxīn. Traditional handicraft techniques are being steadily improved.</defn></entry>
<entry><headword charset="sc">单身狗</headword><headword charset="tc">單身狗</headword><pron type="hypy" tones="numbers">dan1shen1gou3</pron><defn>(Internet slang) person who is neither married nor in a relationship (used self-deprecatingly)</defn></entry>
<entry><headword charset="sc">老弟</headword><headword charset="tc">老弟</headword><pron type="hypy" 

In [96]:
for en in entries_data:
    print(en)

{'simplified': '游戏', 'traditional': '遊戲', 'pinyin': 'yóu xì', 'definition': 'noun recreation; game 做遊戲 Zuò yóuxì play games ', 'examples': ['verb play 孩子們在公園裡遊戲。 Háizi men zài gōngyuán lǐ yóuxì. The children are playing in the park.']}
{'simplified': '革新', 'traditional': '革新', 'pinyin': 'gé xīn', 'definition': 'noun innovation; renovation 技術革新 jìshù géxīn technological innovation ', 'examples': ['verb innovate; improve 傳統的手工藝技術不斷革新。 Chuántǒng de shǒu gōngyì jìshù bùduàn géxīn. Traditional handicraft techniques are being steadily improved.']}
{'simplified': '单身狗', 'traditional': '單身狗', 'pinyin': 'dān shēn gǒu', 'definition': '(Internet slang) person who is neither married nor in a relationship (used self-deprecatingly)', 'examples': []}
{'simplified': '老弟', 'traditional': '老弟', 'pinyin': 'lǎo dì', 'definition': 'noun [familiar form of address to a man somewhat younger than oneself] young man (or fellow); my boy', 'examples': []}
{'simplified': '本事', 'traditional': '本事', 'pinyin': 'běn s

In [52]:
def tell_gf_she_is_silly():
    """Function to tell your girlfriend she's silly."""
    print("Hey, you're so silly! 😊")

# Call the function
tell_gf_she_is_silly()

Hey, you're so silly! 😊


In [50]:
file_text

'<?xml version="1.0" encoding="UTF-8"?><plecoflash formatversion="2" creator="Pleco User 19097293" generator="Pleco 2.0 Flashcard Exporter" platform="iPhone OS" created="1735597172"><categories></categories><cards><card language="chinese" created="1735513394" modified="1735513394"><entry><headword charset="sc">游戏</headword><headword charset="tc">遊戲</headword><pron type="hypy" tones="numbers">you2xi4</pron><defn>noun recreation; game 做遊戲 Zuò yóuxì play games \nverb play 孩子們在公園裡遊戲。 Háizi men zài gōngyuán lǐ yóuxì. The children are playing in the park.</defn></entry><dictref dictid="PACE" entryid="35050240"/></card><card language="chinese" created="1735514285" modified="1735514285"><entry><headword charset="sc">革新</headword><headword charset="tc">革新</headword><pron type="hypy" tones="numbers">ge2xin1</pron><defn>noun innovation; renovation 技術革新 jìshù géxīn technological innovation \nverb innovate; improve 傳統的手工藝技術不斷革新。 Chuántǒng de shǒu gōngyì jìshù bùduàn géxīn. Traditional handicraft te

In [38]:
FLASCHARD_FILE_ARCHIVE_FOLDER_ID

'1S_r1Y_uGNmwlYmqByyjVa1pnQVC07Plf'

In [42]:
x = get_items_by_name(SERVICE, FLASCHARD_FILE_ARCHIVE_FOLDER, is_folder=True, is_root=True)[0]

In [43]:
x

{'mimeType': 'application/vnd.google-apps.folder',
 'parents': ['0ANOuDrZlca71Uk9PVA'],
 'id': '1S_r1Y_uGNmwlYmqByyjVa1pnQVC07Plf',
 'name': '_Pleco',
 'modifiedTime': '2024-12-30T22:08:18.038Z'}

In [45]:
from datetime import datetime

modified_time = datetime.fromisoformat(x['modifiedTime'].replace('Z', '+00:00'))
modified_time

datetime.datetime(2024, 12, 30, 22, 8, 18, 38000, tzinfo=datetime.timezone.utc)

In [26]:
results = SERVICE.files().list(
    q="'root' in parents and name = 'pleco_flashcards.xml' and mimeType != 'application/vnd.google-apps.folder'",  # Only get items in the root directory
    fields="nextPageToken, files(id, name, mimeType, parents)"
).execute()

In [27]:

items = results.get('files', [])

In [28]:
items

[{'mimeType': 'application/xml',
  'parents': ['0ANOuDrZlca71Uk9PVA'],
  'id': '18FY69M7X4WbjrsE1J1bLtdgNW4BtLUWl',
  'name': 'pleco_flashcards.xml'}]

In [ ]:

if __name__ == "__main__":
    # Replace 'your_target_folder_id' with the actual folder ID where files should be moved
    target_folder_id = 'your_target_folder_id'
    monitor_google_drive(target_folder_id, interval=300)  # Check every 5 minutes


In [97]:
import requests

# AnkiConnect API endpoint
ANKI_CONNECT_URL = "http://localhost:8765"

# Request payload to retrieve cards from the "Pleco Import" deck
payload = {
    "action": "findCards",
    "version": 6,
    "params": {
        "query": 'deck:"Pleco Import"'  # Query for cards only in the "Pleco Import" deck
    }
}

# Send the request
response = requests.post(ANKI_CONNECT_URL, json=payload)

# Parse the response
if response.status_code == 200:
    result = response.json()
    if 'result' in result:
        card_ids = result['result']
        print(f"Total cards found in 'Pleco Import': {len(card_ids)}")
        # print("Card IDs:", card_ids)
    else:
        print("Error:", result.get('error'))
else:
    print("Failed to connect to AnkiConnect. Ensure Anki is running and AnkiConnect is installed.")

Total cards found in 'Pleco Import': 1659


In [98]:
card_ids

[1672167816116,
 1673620992646,
 1673621037811,
 1673621079276,
 1673621083845,
 1673621114815,
 1673621154054,
 1673622350427,
 1673622502227,
 1673622623609,
 1673622627759,
 1673624247073,
 1673629457192,
 1673629471151,
 1673629483287,
 1673629492574,
 1673629505902,
 1673631635645,
 1673631685822,
 1673631762292,
 1673645161076,
 1673708042261,
 1673708096706,
 1673729708428,
 1673729746289,
 1673744336098,
 1673759116989,
 1673759265046,
 1673759269066,
 1673791452312,
 1673791792982,
 1673792574465,
 1673792672349,
 1673882232000,
 1673994557006,
 1673994560299,
 1673994611445,
 1673994676512,
 1674142340992,
 1674148554648,
 1674148557747,
 1674148563305,
 1674148568080,
 1674148587922,
 1674148656781,
 1674148659637,
 1674148676699,
 1674162670636,
 1674229284302,
 1674229507481,
 1674229927982,
 1674486117479,
 1674486131834,
 1674486152757,
 1674486201120,
 1674486233510,
 1674486293126,
 1674576894993,
 1674577013521,
 1674577248324,
 1674765988201,
 1675194942873,
 1675194

In [108]:
import requests

# AnkiConnect API endpoint
ANKI_CONNECT_URL = "http://localhost:8765"

flashcard_set = set()

# Step 1: Find all card IDs in the "Pleco Import" deck
find_payload = {
    "action": "findCards",
    "version": 6,
    "params": {
        "query": 'deck:"Pleco Import"'  # Query for cards only in the "Pleco Import" deck
    }
}

response = requests.post(ANKI_CONNECT_URL, json=find_payload)

if response.status_code == 200:
    find_result = response.json()
    if 'result' in find_result and find_result['result']:
        card_ids = find_result['result']
        print(f"Total cards found in 'Pleco Import': {len(card_ids)}")

        # Step 2: Retrieve detailed information for these card IDs
        info_payload = {
            "action": "cardsInfo",
            "version": 6,
            "params": {
                "cards": card_ids  # Pass the list of card IDs
            }
        }

        info_response = requests.post(ANKI_CONNECT_URL, json=info_payload)

        if info_response.status_code == 200:
            info_result = info_response.json()
            if 'result' in info_result and info_result['result']:
                # Step 3: Extract the "Front" field for each card and add to the set
                for card in info_result['result']:
                    card_id = card['cardId']
                    front_field = re.sub(r'[^\u4e00-\u9fff]', '', card['fields']['Front']['value'])  # Filter out non-Chinese characters
                    flashcard_set.add(front_field)
                print("success")
            else:
                print("Error retrieving card information:", info_result.get('error'))
        else:
            print("Failed to connect to AnkiConnect for card information.")
    else:
        print("No cards found or error:", find_result.get('error'))
else:
    print("Failed to connect to AnkiConnect for finding cards.")


Total cards found in 'Pleco Import': 1659
success


In [110]:
len(flashcard_set)

1655

In [111]:
"陳" in flashcard_set

True

In [109]:
flashcard_set

{'打包',
 '帶路',
 '陪伴',
 '師長',
 '金錢',
 '注視',
 '町',
 '設',
 '騎樓',
 '裏',
 '章節',
 '預覽',
 '作品',
 '醉',
 '栗子',
 '單字',
 '貝',
 '聘用',
 '法力',
 '寶可夢',
 '欸',
 '點著',
 '條件',
 '對象',
 '瞬',
 '顯示',
 '減',
 '拜訪',
 '桃子',
 '猶',
 '筋',
 '長得',
 '有待',
 '厚',
 '真相',
 '將近',
 '煙',
 '書局',
 '身為',
 '略',
 '國民黨',
 '搜尋',
 '學區',
 '營養',
 '導',
 '資料',
 '訊息',
 '巷',
 '評論',
 '卷',
 '風景',
 '雙倍',
 '過分',
 '翻然',
 '煎',
 '訂位',
 '仍然',
 '拋棄',
 '愉悅',
 '副',
 '何',
 '教導',
 '將棋',
 '規定',
 '簽名',
 '民國',
 '輸出',
 '劃',
 '突如其來',
 '占領',
 '體力',
 '販毒',
 '頻率',
 '辯解',
 '盡力',
 '舖',
 '神奇',
 '進階',
 '替',
 '堪',
 '探索',
 '繞路',
 '陌生',
 '菜脯',
 '立即',
 '攀岩',
 '引',
 '隻',
 '總之',
 '嗚',
 '所需',
 '布',
 '王牌',
 '要',
 '列入',
 '耐',
 '無從',
 '菇',
 '抽獎',
 '專',
 '集合',
 '描述',
 '二手',
 '黑桃',
 '不愧',
 '時時',
 '明辨',
 '襲擊',
 '喜悅',
 '立法',
 '晶晶',
 '榛',
 '屋',
 '骨頭',
 '原名',
 '反抗',
 '是以',
 '拈',
 '內涵意義',
 '愚人節',
 '未來',
 '化',
 '命名',
 '有著',
 '號召',
 '走道',
 '晚',
 '亂',
 '盆子',
 '能力',
 '平日',
 '摔倒',
 '難免',
 '賽',
 '辦案',
 '瘋',
 '觀音',
 '詹',
 '鬆',
 '人物',
 '剪',
 '時鐘',
 '均勻',
 '旱災',
 '以',
 '排除',
 '近處',
 '寶貴',
